Use `c2gsea` environment here

In [13]:
import os
import pandas as pd
import numpy as np

import scanpy as sc
import anndata as ad
import gseapy as gp

import re
from collections import defaultdict
import pickle
import glob

In [2]:
msigdb_path = "/home/ddz5/Desktop/c2s-RL/gene_programs_dev/clustering/msigdb.tsv"

In [5]:
gene_sets = pd.read_csv(msigdb_path, sep='\t')

### Basic sanity check for gene programs across cell types of a dataset

In [18]:
# let's just load in one dataset for now
# we also need to load in the GSEA gene sets

gene_set_data_path = "/home/ddz5/Desktop/c2s-RL/gene_programs_dev/gene_set_data"

dataset_dir = os.path.join(gene_set_data_path, "local(191)")
dataset_path = os.path.join(dataset_dir, "top_gene_programs.csv")

In [19]:
gps_df = pd.read_csv(dataset_path)

In [14]:
# let's calculate several statistics - what percentage of gene programs for a given cell type contain the cell type name
# what percentage of all gene programs which contain the cell type name are present in this dataset
# let's also store raw counts in case one cell type has an inflated number of associated gene sets

# recall: our dataframe is set up with top 100 gene sets followed by bottom 25 gene sets (prob. not useful) for each cell type

# mapping between cell type to common gene sets (i.e. gene sets which contain the cell type name)

gps_df['Cell Type'].unique()

array(['endothelial cell', 'epithelial cell of proximal tubule',
       'fibroblast', 'kidney distal convoluted tubule epithelial cell',
       'kidney loop of Henle thick ascending limb epithelial cell',
       'kidney loop of Henle thin ascending limb epithelial cell',
       'leukocyte', 'mesangial cell', 'parietal epithelial cell',
       'podocyte', 'renal alpha-intercalated cell',
       'renal beta-intercalated cell', 'renal principal cell'],
      dtype=object)

In [15]:
gene_sets = list(gsea_data['gene_set'])

In [16]:
# we need a way to efficiently generate keywords which we can check against
def extract_keywords(name):
    # filler words which we won't search for
    stopwords = {'cell', 'of', 'the', 'and', 'type'}
    return {word.lower() for word in re.split(r'\s+', name) if word.lower() not in stopwords}

In [17]:
def build_celltype_to_geneset_map(cell_types, all_gene_sets):
    celltype_to_genesets = defaultdict(set)

    # Preprocess gene sets to lowercase
    all_gene_sets_lower = [(gs, gs.lower()) for gs in all_gene_sets]

    for ct in cell_types:
        keywords = extract_keywords(ct)
        for gs, gs_lower in all_gene_sets_lower:
            if any(kw in gs_lower for kw in keywords):
                celltype_to_genesets[ct].add(gs)
    return celltype_to_genesets

In [24]:
gps_df

,Disease,Cell Type,Gene Program,Mean Activity Score,Rank Type
0,normal,endothelial cell,LAKE_ADULT_KIDNEY_C22_ENDOTHELIAL_CELLS_GLOMER...,166.204457,Top
1,normal,endothelial cell,GAVISH_3CA_METAPROGRAM_FIBROBLASTS_CAF_4,127.618750,Top
2,normal,endothelial cell,GOBP_POSITIVE_REGULATION_OF_MYOBLAST_PROLIFERA...,122.356888,Top
3,normal,endothelial cell,GAVISH_3CA_MALIGNANT_METAPROGRAM_41_UNASSIGNED,121.097780,Top
4,normal,endothelial cell,GSE32986_UNSTIM_VS_GMCSF_AND_CURDLAN_HIGHDOSE_...,120.334617,Top
...,...,...,...,...,...
3245,type 2 diabetes mellitus,renal principal cell,ACOSTA_PROLIFERATION_INDEPENDENT_MYC_TARGETS_DN,0.000000,Bottom
3246,type 2 diabetes mellitus,renal principal cell,ACOSTA_PROLIFERATION_INDEPENDENT_MYC_TARGETS_UP,0.000000,Bottom
3247,type 2 diabetes mellitus,renal principal cell,ADDYA_ERYTHROID_DIFFERENTIATION_BY_HEMIN,0.000000,Bottom
3248,type 2 diabetes mellitus,renal principal cell,AFFAR_YY1_TARGETS_DN,0.000000,Bottom


In [52]:
def compute_statistics(df, all_gene_sets):
    df = df[df['Rank Type'] == 'Top'].copy()

    cell_types = df['Cell Type'].unique()
    celltype_to_genesets = build_celltype_to_geneset_map(cell_types, all_gene_sets)

    stats = []

    for (disease, cell_type), group in df.groupby(['Disease', 'Cell Type']):
        keywords = extract_keywords(cell_type)
        # print(cell_type, keywords)
        # take top 100
        group_gene_programs = set(group['Gene Program'].unique())
        
        # gene programs in the dataset that contain any of the keywords
        matched_gene_programs = {
            gp for gp in group_gene_programs
            if any(kw in gp.lower() for kw in keywords)
        }

        global_matches = celltype_to_genesets[cell_type]

        # Compute stats
        total_gp_in_dataset = len(group_gene_programs)
        matching_gp_in_dataset = len(matched_gene_programs)
        total_global_matches = len(global_matches)
        overlapping_gp = len(group_gene_programs & global_matches)

        stats.append({
            'disease': disease,
            'cell_type': cell_type,
            'num_gene_programs_in_dataset': total_gp_in_dataset,
            'num_matching_gene_programs_in_dataset': matching_gp_in_dataset,
            'num_global_matching_gene_sets': total_global_matches,
            'num_global_matching_gene_sets_present': overlapping_gp,
            '%_containing_keywords_in_dataset': 100 * matching_gp_in_dataset / total_gp_in_dataset if total_gp_in_dataset > 0 else 0.0,
            '%_of_global_matches_present': 100 * overlapping_gp / total_global_matches if total_global_matches > 0 else 0.0
        })

    return pd.DataFrame(stats)

In [53]:
final_stats = compute_statistics(gps_df, gene_sets)

In [ ]:
final_stats

### Pseudo-bulk Approach
Not sure, how useful the above is. Perhaps we can just run GSEA on our transformed pseuduo-bulk data

In [22]:
# recall this is our method's results
gps_df

,Disease,Cell Type,Gene Program,Mean Activity Score,Rank Type
0,normal,endothelial cell,LAKE_ADULT_KIDNEY_C22_ENDOTHELIAL_CELLS_GLOMER...,166.204457,Top
1,normal,endothelial cell,GAVISH_3CA_METAPROGRAM_FIBROBLASTS_CAF_4,127.618750,Top
2,normal,endothelial cell,GOBP_POSITIVE_REGULATION_OF_MYOBLAST_PROLIFERA...,122.356888,Top
3,normal,endothelial cell,GAVISH_3CA_MALIGNANT_METAPROGRAM_41_UNASSIGNED,121.097780,Top
4,normal,endothelial cell,GSE32986_UNSTIM_VS_GMCSF_AND_CURDLAN_HIGHDOSE_...,120.334617,Top
...,...,...,...,...,...
3245,type 2 diabetes mellitus,renal principal cell,ACOSTA_PROLIFERATION_INDEPENDENT_MYC_TARGETS_DN,0.000000,Bottom
3246,type 2 diabetes mellitus,renal principal cell,ACOSTA_PROLIFERATION_INDEPENDENT_MYC_TARGETS_UP,0.000000,Bottom
3247,type 2 diabetes mellitus,renal principal cell,ADDYA_ERYTHROID_DIFFERENTIATION_BY_HEMIN,0.000000,Bottom
3248,type 2 diabetes mellitus,renal principal cell,AFFAR_YY1_TARGETS_DN,0.000000,Bottom


In [17]:
# plan: generate pseudo bulk to run GSEA analysis, compare results to our model results

# method to compare: 
# remember: we obtain the .h5ad path from the training_inputs dataset path (however, they are all on transfer partition, I copied
# over a dataset from partition for use in notebook before writing a script which we can run on transfer)


### NOT NEEDED THEN:
# training_input_path = "/home/ddz5/scratch/Cell2GSEA_QA_dataset_models/finished_datasets/local(191)/training_inputs.pickle"

# # training_input_path = "/home/ddz5/scratch/Cell2GSEA_QA_dataset_models/set_7/local(302)/new_training_inputs.pickle"

# with open(training_input_path, 'rb') as f:
#     train_inputs = pickle.load(f)


local191 = sc.read_h5ad("/home/ddz5/scratch/Cell2GSEA_QA_dataset_models/local(191)_colunified.h5ad")

/gpfs/radev/home/ddz5/.conda/envs/c2gsea_clean/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [20]:
local191.obs['cell_type'].unique()

['epithelial cell of proximal tubule', 'kidney loop of Henle thick ascending limb epi..., 'kidney distal convoluted tubule epithelial cell', 'renal alpha-intercalated cell', 'renal beta-intercalated cell', ..., 'podocyte', 'renal principal cell', 'mesangial cell', 'fibroblast', 'leukocyte']
Length: 13
Categories (13, object): ['fibroblast', 'endothelial cell', 'mesangial cell', 'podocyte', ..., 'parietal epithelial cell', 'kidney distal convoluted tubule epithelial cell', 'kidney loop of Henle thick ascending limb epi..., 'kidney loop of Henle thin ascending limb epit...]

In [21]:
# groupby = ['cell_type', 'disease', 'sex', 'tissue']

# only grouping by cell_type and disease for now (following our method)
groupby = ['cell_type', 'disease']

local191.obs['group'] = local191.obs[groupby[0]].astype(str) + "_" + local191.obs[groupby[1]].astype(str)

In [49]:
pseudobulk_df = local191.to_df().groupby(local191.obs["group"]).sum()
min_expr_genes = (pseudobulk_df > 0).sum(axis=0) >= 1
pseudobulk_df = pseudobulk_df.loc[:, min_expr_genes]

In [50]:
# (groups, # genes)
pseudobulk_df.shape

(26, 29221)

In [44]:
# iterating through each group and running GSEA compared to rest
target_group = list(pseudobulk_df.index)[1]
target_group

'endothelial cell_type 2 diabetes mellitus'

In [51]:
# take target_group expr, compare to all other cells
expr_target = pseudobulk_df.loc[target_group]
# all other cells
expr_rest = pseudobulk_df.drop(index=target_group).mean(axis=0)

In [52]:
# ranking genes by log2fc
log2fc = (expr_target + 1).apply(np.log2) - (expr_rest + 1).apply(np.log2)
ranked_genes = log2fc.sort_values(ascending=False)

In [53]:
# let's iterate through all the .gmt files to calculate GSEA across all MSigDB gene collections
msigdb_gmt_dir = "/home/ddz5/Desktop/c2s-RL/gene_programs_dev/gene_set_data/msigdb_v2024.1.Hs_GMTs"
# only want gene set collections with gene symbols (not Entrez)
gmt_files = glob.glob(os.path.join(msigdb_gmt_dir, "*.symbols.gmt"))

# only keep full collections
full_collection_gmts = [f for f in gmt_files if ".all." in os.path.basename(f)]

# also keep h.all and c5.go
# full_collection_gmts += [
#     f for f in gmt_files 
#     if any(x in os.path.basename(f) for x in ["h.all.", "c5.go."]) 
#     and f not in full_collection_gmts
# ]

# Remove duplicates just in case
full_collection_gmts = list(set(full_collection_gmts))

print("Using the following full-collection .gmt files:")
for f in full_collection_gmts:
    print("-", os.path.basename(f))

Using the following full-collection .gmt files:
- c1.all.v2024.1.Hs.symbols.gmt
- c4.all.v2024.1.Hs.symbols.gmt
- c2.all.v2024.1.Hs.symbols.gmt
- h.all.v2024.1.Hs.symbols.gmt
- c3.all.v2024.1.Hs.symbols.gmt
- c7.all.v2024.1.Hs.symbols.gmt
- c6.all.v2024.1.Hs.symbols.gmt
- c8.all.v2024.1.Hs.symbols.gmt
- c5.all.v2024.1.Hs.symbols.gmt


In [54]:
gsea_results = {}  # Key: collection name, Value: GSEApy result object

# just run on one to see output
for gmt_path in full_collection_gmts[:1]:
    base_name = os.path.basename(gmt_path).replace(".symbols.gmt", "")
    print(f"🚀 Running GSEA for: {base_name}")

    try:
        enr = gp.prerank(
            rnk=ranked_genes,
            gene_sets=gmt_path,
            permutation_num=100,
            seed=42,
            verbose=False,
            outdir=None,     # Don't save any files
            format=None,     # Don't generate plots
        )
        gsea_results[base_name] = enr

    except Exception as e:
        print(f"Failed to run GSEA for {base_name}: {e}")

2025-04-05 17:37:54,532 [WARNING] Duplicated values found in preranked stats: 48.61% of genes
The order of those genes will be arbitrary, which may produce unexpected results.


🚀 Running GSEA for: c1.all.v2024.1.Hs


In [55]:
gsea_results['c1.all.v2024.1.Hs'].res2d.iloc[:10]

,Name,Term,ES,NES,NOM p-val,FDR q-val,FWER p-val,Tag %,Gene %,Lead_genes
0,prerank,chrYq11,0.353723,2.003486,0.0,0.0,0.0,23/23,64.66%,PRY;DAZ3;RPS4Y2;HSFY1;FAM224A;HSFY2;DAZ1;TTTY1...
1,prerank,chr2q34,-0.637107,-1.661438,0.0,0.099124,0.1,10/19,15.86%,ACADL;PTH2R;ERBB4;VWC2L;LANCL1-AS1;CPS1;UNC80;...
2,prerank,chr3q28,-0.628631,-1.661351,0.0,0.049562,0.1,8/19,12.99%,TMEM207;P3H2-AS1;CLDN16;P3H2;TPRG1-AS1;CLDN1;G...
3,prerank,chr2q22,-0.645655,-1.637577,0.0,0.046258,0.12,7/17,12.12%,THSD7B;TEX41;NXPH2;LRP1B;KYNU;HNMT;ZEB2-AS1
4,prerank,chr2q12,-0.565418,-1.626741,0.0,0.044606,0.15,22/41,23.61%,SLC9A4;SULT1C2;IL1RL2;LINC01594;LINC01886;SLC9...
5,prerank,chr9q32,-0.594006,-1.622627,0.0,0.037667,0.16,21/32,26.69%,WHRN;TNFSF15;KIAA1958;KIF12;RNF183;COL27A1;BSP...
6,prerank,chr2q24,-0.531798,-1.609004,0.0,0.042954,0.21,23/53,18.46%,KCNH7;KCNJ3;FIGN;SCN7A;SCN2A;GRB14;CCDC148;UPP...
7,prerank,chr1p12,-0.608781,-1.588534,0.0,0.049562,0.26,11/20,18.05%,HAO2;NOTCH2;SPAG17;VTCN1;HMGCS2;PHGDH;ZNF697;L...
8,prerank,chrXp21,-0.590723,-1.538103,0.020408,0.074343,0.43,9/22,26.23%,GK;DMD;IL1RAPL1;XK;CFAP47;LANCL3;CYBB;PRRG1;TAB3
9,prerank,chr2q36,-0.538516,-1.505863,0.01,0.095819,0.57,14/35,15.61%,PID1;SLC19A3;COL4A3;COL4A4;MOGAT1;DOCK10;SGPP2...


In [33]:
gsea_results['c1.all.v2024.1.Hs'].res2d.iloc[:10]

,Name,Term,ES,NES,NOM p-val,FDR q-val,FWER p-val,Tag %,Gene %,Lead_genes
0,prerank,chr9q13,0.925943,3.133514,0.0,0.0,0.0,2/38,7.40%,LINC01410;LINC01189
1,prerank,chr9p11,0.60018,1.638473,0.0,0.0,0.0,3/31,3.62%,BMS1P14;FOXD4L6;CNTNAP3B
2,prerank,chr1p12,-0.754577,-1.430592,0.0,0.019793,0.02,18/44,23.51%,HMGCS2;SPAG17;HSD3B2;NOTCH2;HAO2;ZNF697;LINC01...
3,prerank,chr9p23,-0.854408,-1.426044,0.01087,0.009896,0.02,4/16,3.82%,LURAP1L;TYRP1;PTPRD;LINC01235
4,prerank,chr3p26,-0.755405,-1.39643,0.0,0.032988,0.1,18/40,18.40%,GRM7;LRRN1;CHL1;GRM7-AS1;LINC01266;CNTN4;EGOT;...
5,prerank,chr2q22,-0.730361,-1.380103,0.0,0.044534,0.18,9/46,9.61%,THSD7B;NXPH2;TEX41;ZEB2-AS1;LRP1B;HNMT;ARHGAP1...
6,prerank,chrXq11,-0.775147,-1.377826,0.0,0.035627,0.18,7/24,20.64%,AMER1;ZC3H12B;ASB12;ARHGEF9;ZC4H2;MTMR8;LINC01278
7,prerank,chr3p12,-0.715643,-1.369309,0.0,0.044534,0.25,10/61,15.13%,ROBO2;LINC00506;CADM2;LINC02027;CNTN3;VGLL3;GB...
8,prerank,chr2q34,-0.736697,-1.368767,0.0,0.038172,0.25,14/37,15.52%,ERBB4;VWC2L;UNC80;ACADL;CPS1;PTH2R;SPAG16;IKZF...
9,prerank,chr4q25,-0.684966,-1.360229,0.0,0.04206,0.31,21/71,19.95%,EGF;LEF1;SGMS2;ELOVL6;ENPEP;LEF1-AS1;PLA2G12A;...


In [ ]:
# TODO: let's calculate how many gene sets that our method predicts also show up in GSEA analysis